# Crypto Connection Test — Alpaca Paper Trading

**Purpose:** Validate end-to-end order flow on Alpaca paper trading using crypto (BTC/USD, ETH/USD).  
Crypto trades 24/7, so this test works any time — no market hours dependency.

**Test sequence:**
1. Connect to Alpaca in LIVE mode (paper account)
2. Fetch live crypto prices
3. Buy $25 of BTC → verify fill
4. Check position appears
5. Sell position → verify fill
6. Round-trip summary (entry, exit, latency, P&L)

In [1]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 1: IMPORTS & CONNECTION
# ══════════════════════════════════════════════════════════════════════════════

import sys, os, time, logging
from pathlib import Path
from datetime import datetime, timedelta

import pandas as pd
import numpy as np

# Ensure src/ is on the path
src_dir = Path.cwd() if Path.cwd().name == 'src' else Path.cwd() / 'src'
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

from connection.alpaca_connection import AlpacaConfig, AlpacaConnection, TradingMode

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(name)s | %(levelname)s | %(message)s',
    datefmt='%H:%M:%S'
)

# ─── Connect in LIVE mode (paper account — real orders, fake money) ─────────
config = AlpacaConfig.from_env()
config.trading_mode = TradingMode.LIVE  # Must be LIVE to submit real orders

conn = AlpacaConnection(config)
conn.test_connection()

print(f"\n⚠️  Mode: LIVE (paper) — orders WILL be submitted to your paper account")
print(f"    Crypto trades 24/7 — no market hours restriction")

05:04:28 | connection.alpaca_connection | INFO | Trading client initialized (paper=True, mode=live)


  ALPACA CONNECTION TEST
  Status:          AccountStatus.ACTIVE
  Mode:            PAPER
  Trading Mode:    LIVE
  Cash:            $1,000,000.00
  Buying Power:    $2,000,000.00
  Portfolio Value: $1,000,000.00
  Market Open:     False
  Day Trades:      0
  PDT Flag:        False

⚠️  Mode: LIVE (paper) — orders WILL be submitted to your paper account
    Crypto trades 24/7 — no market hours restriction


In [2]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 2: FETCH LIVE CRYPTO PRICES
# ══════════════════════════════════════════════════════════════════════════════

from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest, CryptoLatestBarRequest
from alpaca.data.timeframe import TimeFrame

CRYPTO_SYMBOLS = ['BTC/USD', 'ETH/USD']

crypto_client = conn.crypto_data_client

# ─── Latest bars (current prices) ──────────────────────────────────────────
print("📡 Fetching live crypto prices...\n")

latest_request = CryptoBarsRequest(
    symbol_or_symbols=CRYPTO_SYMBOLS,
    timeframe=TimeFrame.Minute,
    start=datetime.now() - timedelta(minutes=5),
)
latest_bars = crypto_client.get_crypto_bars(latest_request)

crypto_prices = {}
for symbol in CRYPTO_SYMBOLS:
    bars = latest_bars[symbol]
    if bars:
        last_bar = bars[-1]
        crypto_prices[symbol] = {
            'price': float(last_bar.close),
            'volume': float(last_bar.volume),
            'timestamp': str(last_bar.timestamp),
        }
        print(f"  {symbol:8s}  ${float(last_bar.close):>12,.2f}  "
              f"vol: {float(last_bar.volume):.4f}  @ {last_bar.timestamp}")
    else:
        print(f"  {symbol:8s}  ⚠️  No data")

# ─── Recent history (last 24h for context) ────────────────────────────────
print(f"\n📊 24-Hour History:")
history_request = CryptoBarsRequest(
    symbol_or_symbols=CRYPTO_SYMBOLS,
    timeframe=TimeFrame.Hour,
    start=datetime.now() - timedelta(hours=24),
)
history_bars = crypto_client.get_crypto_bars(history_request)

for symbol in CRYPTO_SYMBOLS:
    bars = history_bars[symbol]
    if bars and len(bars) >= 2:
        open_price = float(bars[0].open)
        close_price = float(bars[-1].close)
        high = max(float(b.high) for b in bars)
        low = min(float(b.low) for b in bars)
        change_pct = (close_price - open_price) / open_price * 100
        print(f"  {symbol:8s}  24h: {change_pct:+.2f}%  "
              f"H: ${high:,.2f}  L: ${low:,.2f}  "
              f"({len(bars)} hourly bars)")

print(f"\n✅ Crypto data feed is working")

05:04:33 | connection.alpaca_connection | INFO | Crypto data client initialized


📡 Fetching live crypto prices...

  BTC/USD   $   68,465.71  vol: 0.0000  @ 2026-02-17 05:03:00+00:00
  ETH/USD   $    1,988.00  vol: 0.0000  @ 2026-02-17 05:03:00+00:00

📊 24-Hour History:
  BTC/USD   24h: +0.14%  H: $70,094.63  L: $65,471.07  (24 hourly bars)
  ETH/USD   24h: +1.62%  H: $2,037.38  L: $1,937.54  (24 hourly bars)

✅ Crypto data feed is working


In [3]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 3: BUY $25 OF BTC — ROUND-TRIP TEST
# ══════════════════════════════════════════════════════════════════════════════
# Uses notional order ($25) so we don't need to calculate fractional BTC qty.
# Alpaca paper trading supports fractional crypto.

TEST_SYMBOL = 'BTC/USD'
TEST_AMOUNT = 25.0  # Dollar amount to buy

print(f"{'═'*60}")
print(f"  CRYPTO ROUND-TRIP TEST")
print(f"  Symbol: {TEST_SYMBOL}  |  Amount: ${TEST_AMOUNT}")
print(f"{'═'*60}")

# ─── Step 1: Record pre-trade state ─────────────────────────────────────────
pre_account = conn.get_account()
pre_positions = conn.get_positions()
pre_cash = pre_account['cash']

# Check if we already hold this crypto
existing_crypto = [p for p in pre_positions if p['symbol'] == TEST_SYMBOL.replace('/', '')]
if existing_crypto:
    print(f"\n⚠️  Already holding {TEST_SYMBOL}: {existing_crypto[0]}")
    print(f"    Close this position first, or change TEST_SYMBOL.")
else:
    print(f"\n  Pre-trade cash: ${pre_cash:,.2f}")
    print(f"  Pre-trade positions: {len(pre_positions)}")

# ─── Step 2: Submit BUY order ──────────────────────────────────────────────
print(f"\n📤 Submitting BUY ${TEST_AMOUNT} of {TEST_SYMBOL}...")

t_buy_start = time.perf_counter()
buy_order = conn.submit_crypto_order(
    symbol=TEST_SYMBOL,
    notional=TEST_AMOUNT,
    side='buy',
)
t_buy_end = time.perf_counter()
buy_latency_ms = (t_buy_end - t_buy_start) * 1000

print(f"  Order ID:    {buy_order['id']}")
print(f"  Status:      {buy_order['status']}")
print(f"  Latency:     {buy_latency_ms:.0f}ms")

# ─── Step 3: Wait for fill and poll ─────────────────────────────────────────
# Paper trading fills are usually instant, but poll just in case
print(f"\n⏳ Waiting for fill...")
buy_filled = None
for attempt in range(10):
    time.sleep(1)
    orders = conn.get_orders(status='all', limit=5)
    for o in orders:
        if o['id'] == buy_order['id']:
            if o['status'] == 'filled':
                buy_filled = o
                break
            elif o['status'] in ('canceled', 'expired', 'rejected'):
                print(f"  ❌ Order {o['status']}: check Alpaca dashboard")
                break
    if buy_filled:
        break
    print(f"  Polling... attempt {attempt + 1}/10 (status: {o.get('status', '?')})")

if buy_filled:
    entry_price = buy_filled['filled_avg_price']
    entry_qty = float(buy_filled.get('qty') or 0)
    print(f"\n  ✅ BUY FILLED")
    print(f"     Price:    ${entry_price:,.2f}")
    print(f"     Qty:      {entry_qty:.8f} {TEST_SYMBOL.split('/')[0]}")
    print(f"     Notional: ${entry_price * entry_qty:,.2f}")
else:
    print(f"\n  ⚠️  Order not filled after 10s — check Alpaca dashboard")
    print(f"     Order ID: {buy_order['id']}")
    entry_price = None

05:04:40 | connection.alpaca_connection | INFO | Crypto order submitted: BUY BTC/USD → OrderStatus.PENDING_NEW


════════════════════════════════════════════════════════════
  CRYPTO ROUND-TRIP TEST
  Symbol: BTC/USD  |  Amount: $25.0
════════════════════════════════════════════════════════════

  Pre-trade cash: $1,000,000.00
  Pre-trade positions: 0

📤 Submitting BUY $25.0 of BTC/USD...
  Order ID:    daa098ed-8183-4392-9fc8-e73e04ed0125
  Status:      pending_new
  Latency:     28ms

⏳ Waiting for fill...

  ✅ BUY FILLED
     Price:    $68,504.90
     Qty:      0.00000000 BTC
     Notional: $0.00


In [4]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 4: VERIFY POSITION & SELL
# ══════════════════════════════════════════════════════════════════════════════

# ─── Step 4: Verify position exists ────────────────────────────────────────
print(f"📋 Checking positions...\n")
time.sleep(2)  # Small delay for settlement

positions = conn.get_positions()
crypto_pos = None
for p in positions:
    # Alpaca stores crypto symbols without slash: BTCUSD
    if p['symbol'] == TEST_SYMBOL.replace('/', ''):
        crypto_pos = p
        break

if crypto_pos:
    print(f"  ✅ Position found:")
    print(f"     Symbol:     {crypto_pos['symbol']}")
    print(f"     Qty:        {float(crypto_pos['qty']):.8f}")
    print(f"     Entry:      ${float(crypto_pos['avg_entry_price']):,.2f}")
    print(f"     Current:    ${float(crypto_pos['current_price']):,.2f}")
    print(f"     Unrealized: ${float(crypto_pos['unrealized_pl']):,.4f}")
    print(f"     Market Val: ${float(crypto_pos['market_value']):,.2f}")
else:
    print(f"  ⚠️  No {TEST_SYMBOL} position found")
    print(f"     All positions: {[p['symbol'] for p in positions]}")

# ─── Step 5: Sell the position ──────────────────────────────────────────────
if crypto_pos:
    sell_qty = float(crypto_pos['qty'])
    print(f"\n📤 Submitting SELL {sell_qty:.8f} {TEST_SYMBOL}...")

    t_sell_start = time.perf_counter()
    sell_order = conn.submit_crypto_order(
        symbol=TEST_SYMBOL,
        qty=sell_qty,
        side='sell',
    )
    t_sell_end = time.perf_counter()
    sell_latency_ms = (t_sell_end - t_sell_start) * 1000

    print(f"  Order ID:    {sell_order['id']}")
    print(f"  Status:      {sell_order['status']}")
    print(f"  Latency:     {sell_latency_ms:.0f}ms")

    # Wait for fill
    print(f"\n⏳ Waiting for sell fill...")
    sell_filled = None
    for attempt in range(10):
        time.sleep(1)
        orders = conn.get_orders(status='all', limit=5)
        for o in orders:
            if o['id'] == sell_order['id']:
                if o['status'] == 'filled':
                    sell_filled = o
                    break
                elif o['status'] in ('canceled', 'expired', 'rejected'):
                    print(f"  ❌ Sell order {o['status']}")
                    break
        if sell_filled:
            break
        print(f"  Polling... attempt {attempt + 1}/10")

    if sell_filled:
        exit_price = sell_filled['filled_avg_price']
        print(f"\n  ✅ SELL FILLED")
        print(f"     Price:    ${exit_price:,.2f}")
    else:
        exit_price = None
        print(f"\n  ⚠️  Sell not filled after 10s")
else:
    print(f"\n⏭️  Skipping sell — no position to close")
    exit_price = None

📋 Checking positions...



05:04:50 | connection.alpaca_connection | INFO | Crypto order submitted: SELL BTC/USD → OrderStatus.PENDING_NEW


  ✅ Position found:
     Symbol:     BTCUSD
     Qty:        0.00035683
     Entry:      $68,504.90
     Current:    $68,432.27
     Unrealized: $-0.0259
     Market Val: $24.42

📤 Submitting SELL 0.00035683 BTC/USD...
  Order ID:    57b85530-4714-4ff5-a354-3a3e34645bee
  Status:      pending_new
  Latency:     14ms

⏳ Waiting for sell fill...

  ✅ SELL FILLED
     Price:    $68,441.72


In [5]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 5: ROUND-TRIP SUMMARY
# ══════════════════════════════════════════════════════════════════════════════

post_account = conn.get_account()
post_cash = post_account['cash']

print(f"{'═'*60}")
print(f"  ROUND-TRIP TEST RESULTS")
print(f"{'═'*60}")

print(f"\n  Symbol:        {TEST_SYMBOL}")
print(f"  Test Amount:   ${TEST_AMOUNT:.2f}")

if entry_price and exit_price:
    pnl = (exit_price - entry_price) * entry_qty
    pnl_pct = (exit_price - entry_price) / entry_price * 100
    cash_change = post_cash - pre_cash

    print(f"\n  Entry Price:   ${entry_price:,.2f}")
    print(f"  Exit Price:    ${exit_price:,.2f}")
    print(f"  Quantity:      {entry_qty:.8f}")
    print(f"  P&L:           ${pnl:,.4f} ({pnl_pct:+.4f}%)")
    print(f"  Cash Before:   ${pre_cash:,.2f}")
    print(f"  Cash After:    ${post_cash:,.2f}")
    print(f"  Cash Change:   ${cash_change:,.4f}")
    print(f"\n  Buy Latency:   {buy_latency_ms:.0f}ms")
    print(f"  Sell Latency:  {sell_latency_ms:.0f}ms")
    print(f"  Total Time:    {buy_latency_ms + sell_latency_ms:.0f}ms")

    print(f"\n{'═'*60}")
    print(f"  ✅ ROUND-TRIP COMPLETE — Connection is working!")
    print(f"  Orders fill, positions track, P&L records.")
    print(f"{'═'*60}")
else:
    print(f"\n  ⚠️  Incomplete test — check individual cells above")
    if not entry_price:
        print(f"     Buy order did not fill")
    if not exit_price:
        print(f"     Sell order did not fill")

# ─── Show all recent orders for verification ──────────────────────────────
print(f"\n📋 Recent Orders:")
recent = conn.get_orders(status='all', limit=10)
for o in recent:
    icon = '✅' if o['status'] == 'filled' else '⏳' if o['status'] in ('new', 'accepted') else '❌'
    filled_info = f"@ ${o['filled_avg_price']:,.2f}" if o.get('filled_avg_price') else ''
    print(f"  {icon} {o.get('side', '?'):4s} {o['symbol']:8s} "
          f"qty={o.get('qty', '?')}  {o['status']:10s} {filled_info}")

# ─── Confirm no leftover positions ─────────────────────────────────────────
final_positions = conn.get_positions()
crypto_remaining = [p for p in final_positions
                    if p['symbol'] in ('BTCUSD', 'ETHUSD')]
if crypto_remaining:
    print(f"\n⚠️  Remaining crypto positions:")
    for p in crypto_remaining:
        print(f"     {p['symbol']}: {p['qty']} @ ${float(p['avg_entry_price']):,.2f}")
else:
    print(f"\n✅ No leftover crypto positions — clean state")

════════════════════════════════════════════════════════════
  ROUND-TRIP TEST RESULTS
════════════════════════════════════════════════════════════

  Symbol:        BTC/USD
  Test Amount:   $25.00

  Entry Price:   $68,504.90
  Exit Price:    $68,441.72
  Quantity:      0.00000000
  P&L:           $-0.0000 (-0.0922%)
  Cash Before:   $1,000,000.00
  Cash After:    $999,999.84
  Cash Change:   $-0.1600

  Buy Latency:   28ms
  Sell Latency:  14ms
  Total Time:    42ms

════════════════════════════════════════════════════════════
  ✅ ROUND-TRIP COMPLETE — Connection is working!
  Orders fill, positions track, P&L records.
════════════════════════════════════════════════════════════

📋 Recent Orders:
  ✅ sell BTC/USD  qty=0.000356827  filled     @ $68,441.72
  ✅ buy  BTC/USD  qty=None  filled     @ $68,504.90

✅ No leftover crypto positions — clean state


---

## Optional: ETH/USD Test

Run the same round-trip test with Ethereum to confirm multi-asset order flow.

In [6]:
# ══════════════════════════════════════════════════════════════════════════════
# CELL 6: ETH ROUND-TRIP TEST (OPTIONAL)
# ══════════════════════════════════════════════════════════════════════════════

ETH_SYMBOL = 'ETH/USD'
ETH_AMOUNT = 25.0

print(f"{'═'*60}")
print(f"  ETH ROUND-TRIP TEST")
print(f"  Symbol: {ETH_SYMBOL}  |  Amount: ${ETH_AMOUNT}")
print(f"{'═'*60}")

# ── Buy ──
print(f"\n📤 BUY ${ETH_AMOUNT} of {ETH_SYMBOL}...")
t0 = time.perf_counter()
eth_buy = conn.submit_crypto_order(symbol=ETH_SYMBOL, notional=ETH_AMOUNT, side='buy')
eth_buy_ms = (time.perf_counter() - t0) * 1000
print(f"  Status: {eth_buy['status']}  |  Latency: {eth_buy_ms:.0f}ms")

# Poll for fill
eth_entry = None
for _ in range(10):
    time.sleep(1)
    for o in conn.get_orders(status='all', limit=5):
        if o['id'] == eth_buy['id'] and o['status'] == 'filled':
            eth_entry = o['filled_avg_price']
            eth_qty = float(o.get('qty', 0))
            break
    if eth_entry:
        break

if eth_entry:
    print(f"  ✅ Filled @ ${eth_entry:,.2f}  qty={eth_qty:.6f}")
    
    # ── Sell ──
    time.sleep(2)
    print(f"\n📤 SELL {eth_qty:.6f} {ETH_SYMBOL}...")
    t0 = time.perf_counter()
    eth_sell = conn.submit_crypto_order(symbol=ETH_SYMBOL, qty=eth_qty, side='sell')
    eth_sell_ms = (time.perf_counter() - t0) * 1000
    print(f"  Status: {eth_sell['status']}  |  Latency: {eth_sell_ms:.0f}ms")
    
    eth_exit = None
    for _ in range(10):
        time.sleep(1)
        for o in conn.get_orders(status='all', limit=5):
            if o['id'] == eth_sell['id'] and o['status'] == 'filled':
                eth_exit = o['filled_avg_price']
                break
        if eth_exit:
            break
    
    if eth_exit:
        eth_pnl = (eth_exit - eth_entry) * eth_qty
        print(f"  ✅ Filled @ ${eth_exit:,.2f}")
        print(f"  P&L: ${eth_pnl:,.4f}")
        print(f"\n  ✅ ETH round-trip complete!")
    else:
        print(f"  ⚠️  Sell not filled")
else:
    print(f"  ⚠️  Buy not filled")

05:05:13 | connection.alpaca_connection | INFO | Crypto order submitted: BUY ETH/USD → OrderStatus.PENDING_NEW


════════════════════════════════════════════════════════════
  ETH ROUND-TRIP TEST
  Symbol: ETH/USD  |  Amount: $25.0
════════════════════════════════════════════════════════════

📤 BUY $25.0 of ETH/USD...
  Status: pending_new  |  Latency: 11ms


TypeError: float() argument must be a string or a real number, not 'NoneType'